In [1]:
import os 
import sys

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.preprocess import PrepSystem
from src.train import train
from transformers import AdamWeightDecay
from random import randint

2023-12-04 20:41:21.106863: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 20:41:21.133485: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 20:41:21.133510: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 20:41:21.133527: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 20:41:21.138482: I tensorflow/core/platform/cpu_feature_g

In [3]:
tagset = {
        "O": 0,
        "B-PER": 1,
        "I-PER": 2,
        # "B-ORG": 3,
        # "I-ORG": 4,
        "B-LOC": 5,
        "I-LOC": 6,
        #"B-ANIM": 7,
        #"I-ANIM": 8,
        "B-DIS": 13,
        "I-DIS": 14,
}

In [5]:
from src.train import check_gpus
check_gpus()

2023-12-04 20:41:56.881555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 20:41:56 INFO     [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2023-12-04 20:41:56.885339: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 20:41:56.885501: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
pretrained_model_checkpoint = "distilbert-base-uncased"

learning_rate = 2e-5
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=0.0)

filter_tagset = True

In [8]:

# any limited tagset works
# for example, here we remove "ORG" from the tagset for experiment B
tagset = {
        "O": 0,
        "B-PER": 1,
        "I-PER": 2,
        # "B-ORG": 3,
        # "I-ORG": 4,
        "B-LOC": 5,
        "I-LOC": 6,
        "B-ANIM": 7,
        "I-ANIM": 8,
        "B-DIS": 13,
        "I-DIS": 14,
}

In [9]:
system = PrepSystem(
        labels=tagset,
        pretrained_model_checkpoint=pretrained_model_checkpoint,
        dataset_batch_size=16,
        filter_tagset=filter_tagset,
        language="en",
        # set split_filter to None to load the entire dataset
        split_filter=None,  
    )

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

2023-12-04 20:42:11 INFO     Filtered language by en. 
DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 32908
    })
})
2023-12-04 20:42:11 INFO     Keeping these tags only: ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ANIM', 'I-ANIM', 'B-DIS', 'I-DIS']. All other tags will be set to '0'
2023-12-04 20:42:11 INFO     Making sure all labels have sequential IDs. This can happen if a reduced tagset is chosen
2023-12-04 20:42:11 INFO     Swapping these labels: {14: 8, 13: 7, 8: 6, 7: 5, 6: 4, 5: 3}
2023-12-04 20:42:11 INFO     Modified label to ID: {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-LOC': 3, 'I-LOC': 4, 'B-ANIM': 5, 'I-ANIM': 6, 'B-DIS': 7, 'I-DIS': 8}
2023-12-04 20:42:11 INFO     Modified ID to label: {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-LOC', 4: 'I-LO

In [10]:
system.dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 32908
    })
})

In [11]:
system.get_model()
system.model.summary()

2023-12-04 20:42:11.795169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 20:42:11.795337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 20:42:11.795406: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "tf_distil_bert_for_token_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  6921      
                                                                 
Total params: 66369801 (253.18 MB)
Trainable params: 66369801 (253.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
system.load_tokenizer()
sentence = "This is for when you get caught in a sleep-riot and the sky is out of order"
tokenized = system.tokenizer(sentence.split(" "), truncation=True,
            is_split_into_words=True)
print(tokenized["input_ids"])
system.tokenizer.convert_ids_to_tokens(tokenized["input_ids"])

[101, 2023, 2003, 2005, 2043, 2017, 2131, 3236, 1999, 1037, 3637, 1011, 11421, 1998, 1996, 3712, 2003, 2041, 1997, 2344, 102]


['[CLS]',
 'this',
 'is',
 'for',
 'when',
 'you',
 'get',
 'caught',
 'in',
 'a',
 'sleep',
 '-',
 'riot',
 'and',
 'the',
 'sky',
 'is',
 'out',
 'of',
 'order',
 '[SEP]']

In [13]:
# train a smaller subset of the dataset 
num_examples = 1000
for ds in system.data_split:
    system.dataset[ds] = system.dataset[ds].select(range(num_examples))

In [14]:
system.tokenize_dataset()
system.tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [15]:
sample = system.tokenized_dataset["train"][randint(0, 200)]
print(f"Dataset loaded and tokenized.\nSample: {sample}")
print(f"Decoded: {system.tokenizer.convert_ids_to_tokens(sample['input_ids'])}")

Dataset loaded and tokenized.
Sample: {'tokens': ['Karl', 'Marx', "'s", 'writing', 'on', 'the', 'commodity', 'abstraction', 'recognizes', 'a', 'parallel', 'process', '.'], 'ner_tags': [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 6382, 13518, 1005, 1055, 3015, 2006, 1996, 19502, 24504, 14600, 1037, 5903, 2832, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]}
Decoded: ['[CLS]', 'karl', 'marx', "'", 's', 'writing', 'on', 'the', 'commodity', 'abstraction', 'recognizes', 'a', 'parallel', 'process', '.', '[SEP]']


In [16]:
experiment_name = "my_experiment"
train(
        optimizer=optimizer,
        system=system,
        verbose=1,
        epochs=1,
        
        # True to generate tensorboard logs
        tensorboard_callback=False,

        # True to push to hub (make sure you are logged in)
        # False to store results locally
        push_to_hub_callback=False,

        # False to disable
        early_stopping=True, 
        early_stopping_patience=2,
        experiment_name=experiment_name,
    )

Model: "tf_distil_bert_for_token_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  6921      
                                                                 
Total params: 66369801 (253.18 MB)
Trainable params: 66369801 (253.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-12-04 20:42:27 INFO     None
2023-12-04 20:42:27 INFO     Early stopping with patience of 2
2023-12-04 20:42:27 INFO     Storing model checkpoint at each epoch
2023-12-04 20:42:27 INFO     Fitting model...


62/62 [==============================] - ETA: 0s - loss: 0.7094

/home/snek/rise-assignment-ner-finetune/.venv/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2023-12-04 20:42:49 INFO     
2023-12-04 20:42:49 INFO     Entity: ANIM
2023-12-04 20:42:49 INFO     precision 	 0.0
2023-12-04 20:42:49 INFO     recall 	 0.0
2023-12-04 20:42:49 INFO     f1 	 0.0
2023-12-04 20:42:49 INFO     
2023-12-04 20:42:49 INFO     Entity: DIS
2023-12-04 20:42:49 INFO     precision 	 0.0
2023-12-04 20:42:49 INFO     recall 	 0.0
2023-12-04 20:42:49 INFO     f1 	 0.0
2023-12-04 20:42:49 INFO     
2023-12-04 20:42:49 INFO     Entity: LOC
2023-12-04 20:42:49 INFO     precision 	 0.0
2023-12-04 20:42:49 INFO     recall 	 0.0
2023-12-04 20:42:49 INFO     f1 	 0.0
2023-12-04 20:42:49 INFO     
2023-12-04 20:42:49 INFO     Entity: PER
2023-12-


Epoch 1: saving model to ./distilbert-base-uncased-finetuned-ner-my_experiment/checkpoint_01.model.h5
62/62 [==============================] - 22s 266ms/step - loss: 0.7094 - val_loss: 0.3231 - precision: 0.6480 - recall: 0.2014 - f1: 0.3073 - accuracy: 0.9203


In [17]:
system.dataset["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ANIM', 'I-ANIM', 'B-DIS', 'I-DIS'], id=None), length=-1, id=None)}

In [20]:
from evaluate import evaluator
task_evaluator = evaluator("token-classification")

metric = "seqeval"

results = task_evaluator.compute(
    model_or_pipeline=system.model,
    data=system.dataset["test"],
    tokenizer=system.tokenizer,
    metric=metric,
    device=0,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/home/snek/rise-assignment-ner-finetune/.venv/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
results

{'ANIM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 105},
 'DIS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 47},
 'LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 635},
 'PER': {'precision': 0.7097457627118644,
  'recall': 0.850253807106599,
  'f1': 0.7736720554272517,
  'number': 394},
 'overall_precision': 0.7097457627118644,
 'overall_recall': 0.283657917019475,
 'overall_f1': 0.4053236539624924,
 'overall_accuracy': 0.9186568244319004,
 'total_time_in_seconds': 47.389511141000185,
 'samples_per_second': 21.10171588444232,
 'latency_in_seconds': 0.047389511141000186}